### Gruppo 5.a: Castagnacci Giulia (581749), Giordano Elisabetta (536265)

### Analisi boxcovering

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Funzioni aggiuntive create

### createIntervalTree
Dato un dizionario ordinato crea un intervalTree, ovvero una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto. Utilizzata in: spaceindex, boxcovering

In [3]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

### Creazione albero

In [4]:
t = createIntervalTree(dict)

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]


(1.0,1.0) => [2]

### addIntersection
Aggiunge gli elementi di iterator nell'i-esimo array di covers. Utilizzata in: spaceindex, boxcovering

In [5]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

### Versione iniziale di boxcovering

Funzione che prende in input una matrice (i bounding box), un intero che specifica la coordinata su cui si lavora, e un intervalTrees. La funzione restituisce una matrice che contiene tutte le intersezioni tra bounding box.

In [6]:
function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

boxcovering (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [7]:
@btime boxcovering(bb, 1, t) 

  5.783 μs (82 allocations: 3.02 KiB)


5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [8]:
@code_warntype boxcovering(bb, 1, t) 

MethodInstance for boxcovering(::

Vector{Matrix{Float64}}, ::Int64, ::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64})
  from 

boxcovering(bboxes, index, tree) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_boxcovering.ipynb:1
Arguments
  

#self#::Core.Const(boxcovering)
  bboxes::Vector{Matrix{Float64}}
  index::Int64
  tree::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
Locals
  @_5::Union{Nothing, Tuple{Tuple{Int64, Matrix{Float64}}, Tuple{Int64, Int64}}}
  #10::var"#10#11"
  covers::Vector{Vector{Any}}
  @_8::Union{Nothing, Tuple{IntervalValue{Float64, Array}, Int64}, Tuple{IntervalValue{Float64, Array}, Nothing}}
  @_9::Int64
  boundingbox::Matrix{Float64}
  i::Int64
  iterator::Union{IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}, Vector{IntervalValue{Float64, Array}}}
  extent::Vector{Float64}
  x::IntervalValue{Float64, Array}
Body::Vector{Vector{Any}}


1 ─ 

...

Core.Const(2))::Core.PartialStruct(Tuple{Matrix{Float64}, Int64}, Any[Matrix{Float64}, Core.Const(3)])
│         (boundingbox = Core.getfield(%17, 1))
│   %19 = Core.getfield(%12, 2)::Tuple{Int64, Int64}
│   %20 = Base.getindex(bboxes, i)::Matrix{Float64}
│         (extent = Base.getindex(%20, index, Main.:(:)))
│   %22 = IntervalTrees.intersect::Core.Const(intersect)
│   %23 = Core._apply_iterate(Base.iterate, Main.tuple, extent

)::Tuple{Vararg{Float64}}
│         (iterator = (%22)(tree, %23))
│   %25 = iterator::Union{IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}, Vector{IntervalValue{Float64, Array}}}
│         (@_8 = Base.iterate(%25))
│   %27 = (@_8 === nothing)::Bool
│   %28 = Base.not_int(%27)::Bool
└──       goto #5 if not %28
3 ┄ %30 = @_8::Union{Tuple{IntervalValue{Float64, Array}, Int64}, Tuple{IntervalValue{Float64, Array}, Nothing}}
│         (x = Core.getfield(%30, 1))
│   %32 = Core.getfield(%30, 2)

::Union{Nothing, Int64}
│   %33 = Base.getindex(covers, i)::Vector{Any}
│   %34 = Base.getproperty(x, 

:value)::Array
│         Main.append!(%33, %34)
│         (@_8 = Base.iterate(%25, %32))
│   %37 = (@_8 === nothing)::Bool
│   %38 = Base.not_int(%37)::Bool
└──       goto #5 if not %38
4 ─ 

      goto #3
5 ┄       (@_5 = Base.iterate(%7, %19))
│   %42 = (@_5 === nothing)::Bool
│   %43 = Base.not_int(%42)::Bool
└──       goto #7 if not %43
6 ─       goto #2
7 ┄       return covers



In [9]:
@benchmark boxcovering(bb, 1, t) 

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):   5.350 μs …  3.238 ms  ┊ GC (min … max): 0.00% … 70.68%
 Time  (median):      9.367 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.824 μs ± 43.579 μs  ┊ GC (mean ± σ):  3.36% ±  1.22%

               █                                               
  ▃▄▃▂▄▂▂▂▃▃▂▂▂█▃▂▂▃▅▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  5.35 μs         Histogram: frequency by time        23.1 μs <

 Memory estimate: 3.02 KiB, allocs estimate: 82.

### Versione parallelizzata di boxcovering
abbiamo parallelizzato la funzione boxcovering utilizzando la macro @thread prima del ciclo for, e aggiungendo delle funzioni di supporto: createIntervalTree e addIntersection

In [10]:
function boxcovering2(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcovering2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione parallelizzata

In [11]:
@btime boxcovering2(bb, 1, t)

  8.633 μs (86 allocations: 4.83 KiB)


5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [12]:
@benchmark boxcovering2(bb, 1, t)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.900 μs …  4.285 ms  ┊ GC (min … max): 0.00% … 98.00%
 Time  (median):     11.600 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.563 μs ± 66.853 μs  ┊ GC (mean ± σ):  7.24% ±  1.71%

     ▄█                                                        
  █▇▅██▆▄▃▃▂▃▃▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  8.9 μs          Histogram: frequency by time        40.4 μs <

 Memory estimate: 4.83 KiB, allocs estimate: 86.

## Test

In [13]:
using Test

@testset "boxcovering Tests" begin
    V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
    W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))
    cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    dict = Lar.coordintervals(1,bboxes)
    @test typeof(dict) == OrderedDict{Array{Float64,1},Array{Int64,1}}
    @test length(Lar.coordintervals(1,bboxes)) == 54
    @test length(Lar.coordintervals(2,bboxes)) == 54
    @test length(Lar.coordintervals(3,bboxes)) == 54

    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    @test bboxes == [[0.0 0.0; 0.0 1.0],
    [1.0 1.0; 0.0 1.0],
    [2.0 2.0; 0.0 1.0],
    [0.0 1.0; 0.0 0.0],
    [0.0 1.0; 1.0 1.0],
    [1.0 2.0; 0.0 0.0],
    [1.0 2.0; 1.0 1.0]]
    xboxdict = Dict(
     [0.0, 0.0] => [1],
     [1.0, 1.0] => [2],
     [2.0, 2.0] => [3],
     [0.0, 1.0] => [4, 5],
     [1.0, 2.0] => [6, 7])
    @test xboxdict == Lar.coordintervals(1,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
   @test typeof(xs) ==
    IntervalTrees.IntervalBTree{Float64,
    IntervalValue{Float64,Array},64}
end

Test Summary:     | 

Pass  Total
boxcovering Tests |    7      7


Test.DefaultTestSet("boxcovering Tests", Any[], 7, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_boxcovering.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/tests_boxcovering.png?raw=true)